# Slicer Jupyter using docker

This notebook is shows how views and full application window can be displayed and configured to be used in JupyterLab when Slicer runs in docker or on a remote workstation.

It relies on a remote desktop connection and web proxy set up in the [slicer-notebook docker image](https://github.com/Slicer/SlicerDocker/tree/master/slicer-notebook).

This notebook can be [run in the web browser via Binder](https://mybinder.org/v2/gh/slicer/SlicerNotebooks/master?filepath=05_SlicerDockerNotebook.ipynb) or locally using a Jupyter server started by this command:

    docker run -p 8888:8888 -p49053:49053 -v path/to/my/notebooks:/home/sliceruser/work --rm -ti lassoan/slicer-notebook:latest
    
Notes:
- Replace `path/to/my/notebooks` by the actual local path to your notebook folder.
- After the container is started, open `https://127.0.0.1:8888` page in your web browser and copy-paste the token from the docker container's output.

In [65]:
# Read an image using SimpleITK

import JupyterNotebooksLib as slicernb
import SimpleITK as sitk
import sitkUtils as su

# Load 3D image using SimpleITK
reader = sitk.ImageFileReader()
reader.SetFileName("data/MRBrainTumor1.nrrd")
image = reader.Execute()

In [67]:
# Get the SimpleITK image into the Slicer scene
slicer.mrmlScene.Clear(False)  # clear any previously loaded data from the scene
volumeNode = su.PushVolumeToSlicer(image)

# Show volume in slice views
slicernb.ViewDisplay("FourUp")  # choose a layout where 3 slice views are present
slicer.util.setSliceViewerLayers(background=volumeNode, fit=True)  # show this volume in slice viewers

# Create slice view widgets in the notebook
from ipywidgets import VBox
viewWidgets = VBox([slicernb.ViewSliceWidget('Red'), slicernb.ViewSliceWidget('Yellow'), slicernb.ViewSliceWidget('Green')])
viewWidgets.layout.max_width="400px"
display(viewWidgets)

In [71]:
# Apply some processing and view the udpated results

# Process image
blurFilter = sitk.SmoothingRecursiveGaussianImageFilter()
blurFilter.SetSigma(1.0)
blurredImage = blurFilter.Execute(image)

# Update view widgets (without this, the user would need to move the sliders to get an updated image)
su.PushVolumeToSlicer(blurredImage, targetNode=volumeNode)
for viewWidget in viewWidgets.children:
    viewWidget.sliceView.updateImage()

In [79]:
# Set up application window
app = slicernb.AppWindow()
# Hide patient information from slice view
slicernb.showSliceViewAnnotations(False)

# Show markups toolbar
slicer.modules.markups.toolBarVisible=True

# Show volume in 3D view using volume rendering
slicernb.showVolumeRendering(volumeNode, True)

display(app)

# Click on the toolbar buttons to create markups,
# then click in the viewers to place them.

In [80]:
# Display control point positions in each markup node.
from IPython.display import HTML
for markupsNode in getNodesByClass("vtkMRMLMarkupsNode"):
    display(HTML(f"<h3>Markup: {markupsNode.GetName()}</h3>"))
    display(dataframeFromMarkups(markupsNode))

,label,position.R,position.A,position.S,selected,visible,description
0,L-1,7.043775,20.880253,4.3,True,True,
1,L-2,28.175851,-30.943646,4.3,True,True,


,label,position.R,position.A,position.S,selected,visible,description


,label,position.R,position.A,position.S,selected,visible,description
0,F-1,-32.704653,15.345662,4.300000,True,True,
1,F-2,-10.063143,-23.396476,4.300000,True,True,
2,F-3,-15.233632,0.468500,24.992267,True,True,


In [81]:
# Show full markups module GUI
app.setContents("viewers")
slicer.util.findChild(slicer.util.mainWindow(), "PanelDockWidget").show()
slicer.modules.markups.toolBarVisible=True
selectModule("Markups")

In [82]:
# Show full application GUI
app.setContents("full")

In [83]:
# Create link that shows the application GUI in a new browser tab
from ipywidgets import HTML
HTML(f"""<a href="{slicernb.AppWindow.defaultDesktopUrl()}" target="_blank">
<b>Click here</b> to open application window in a new browser tab.</a>""")

HTML(value='<a href="/desktop/" target="_blank">\n<b>Click here</b> to open application window in a new browse…

In [84]:
# Show only viewers
app.setContents("viewers")

In [85]:
from ipywidgets import Button, HBox
import JupyterNotebooksLib as slicernb
fullButton = Button(description='Full')
fullButton.on_click(lambda b: slicernb.AppWindow().setContents("full"))
viewersButton = Button(description='Viewers')
viewersButton.on_click(lambda b: slicernb.AppWindow().setContents("viewers"))
markupsToolbarToggleButton = Button(description='Markups toolbar')
def toggleMarkupsToolBar(b):
    slicer.modules.markups.toolBarVisible = not slicer.modules.markups.toolBarVisible
markupsToolbarToggleButton.on_click(toggleMarkupsToolBar)
HBox([fullButton, viewersButton, markupsToolbarToggleButton]) 

#### Run this example in your browser using Binder: [![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/slicer/SlicerNotebooks/master?filepath=05_SlicerDockerNotebook.ipynb)